<a href="https://colab.research.google.com/github/LamNguyenNN/generalized-hyperbolic-embeddings/blob/main/Colab/SO(1%2C2)_and_PSL(2%2CR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sympy import *

In [ ]:
import numpy as np

In [ ]:
def psl_so(a,b,c,d):
  det = simplify(a*d - b*c)
  if det < 0:
    raise ValueError('Determinant = {} is not positive'.format(det))
  elif det != 1:
    raise ValueError('Determinant = {} does not equal 1'.format(det))

  A = Matrix([[Rational(1,2) * (a**2 + b**2 + c**2 + d**2), -Rational(1,2) * (a**2 - b**2 + c**2 - d**2), a*b + c*d],
            [-Rational(1,2) * (a**2 + b**2 - c**2 - d**2), Rational(1,2) * (a**2 - b**2 - c**2 + d**2), -a*b + c*d],
            [a*c + b*d, -a*c + b*d, a*d + b*c]])

  g = Matrix([[1,0,0],[0,-1,0], [0,0,-1]])

  assert simplify(A.T * g * A) == g

  return simplify(A)

In [ ]:
def so_psl(A):
  a, b, c, d = symbols('a b c d ')

  x1 = A[0]
  y1 = A[1]
  z1 = A[2]

  x2 = A[3]
  y2 = A[4]
  z2 = A[5]

  x3 = A[6]
  y3 = A[7]
  z3 = A[8]

  eqns = [2*(a**2) - x1 + y1 + x2 - y2, 2*(b**2) - x1 - y1 + x2 + y2, 2*(c**2) - x1 + y1 - x2 + y2, 2*(d**2) - x1 - y1 - x2 - y2,
          2*b*d - x3 - y3, 2*c*d - z1 - z2, 2*a*c - x3 + y3, 2*a*b -  z1 + z2,
          a*d - b*c - 1]

  solns = solve(eqns, a,b,c,d, dict=True)
  assert len(solns) == 2

  def multi_simplify(x):
    return simplify(sqrtdenest(radsimp(x)))

  a = multi_simplify(solns[0][a])
  b = multi_simplify(solns[0][b])
  c = multi_simplify(solns[0][c])
  d = multi_simplify(solns[0][d])

  det = simplify(a*d - b*c)
  assert det > 0

  a = multi_simplify(a / sqrt(det))
  b = multi_simplify(b / sqrt(det))
  c = multi_simplify(c / sqrt(det))
  d = multi_simplify(d / sqrt(det))


  return Matrix([[a, b], [c, d]])

In [ ]:
def CompanionMatrix(poly_coeffs):
  # Returns companion matrix for a polynomial given coefficients listed in ascending order with respect to degree of term

  n = len(poly_coeffs)
  rows = []

  for i in range(n):
    row = [0]*n
    row[i-1] = 1
    row[-1] = poly_coeffs[i]
    rows.append(row)

  return Matrix(rows)

In [ ]:
def ConjugatorToRCF(A):
  e1 = Matrix([[1], [0], [0]])
  return simplify(Matrix([ [e1, A*e1, A* A * e1] ]))

In [ ]:
def Mobius(a,b,c,d, z):
  return radsimp(expand((a*z + b) / (c*z + d)))

In [ ]:
def FixedPoints(a,b,c,d):
  trace = a + d
  det = a*d - b*c
  return radsimp(expand((a - d + sqrt(trace**2 - 4*det)) / (2*c))), radsimp(expand((a - d - sqrt(trace**2 - 4*det)) / (2*c)))

In [ ]:
def RotationalAngle(a,b,c,d):
  z1, z2 = FixedPoints(a,b,c,d)

  if im(z1) > 0:
    fp = z1
  else:
    fp = z2

  x = re(fp)
  y = im(fp)

  angle_cos = expand(c * x + d)
  angle_sin = expand(c * y)

  return angle_cos, angle_sin

In [ ]:
def MobiusPointToPoint(z1, z2):
  a,b = z1.as_real_imag()
  c,d = z2.as_real_imag()

  return simplify(Matrix([[d/b, -(a*d / b) + c], [0, 1]]))

In [ ]:
def Elliptic(angle, fixed_point):
  rot = Matrix([[cos(angle), -sin(angle)], [sin(angle), cos(angle)]])
  conj = MobiusPointToPoint(I, fixed_point)

  result = conj * rot * conj.inv()
  #if result[1] < 0:
  #  result = -result

  return expand(result).applyfunc(radsimp)

In [ ]:
def ValidSecondFixedPoint(init_fixed_point, angle1, angle2, n=1, use_smallest_n=False):
  '''Returns center and radius of (euclidean) circle containing all valid fixed points for a double generated elliptic Fuchsian group given a fixed point and angle of the first generator and angle of the second generator'''

  smallest_valid_n = Integer(pi/acos(cos(angle1 + angle2 + pi))) + 1
  if use_smallest_n:
    n = smallest_valid_n
    print('Using smallest valid n={}'.format(smallest_valid_n))
  else:
    if n < smallest_valid_n:
      print('Provided value of n = {} smaller than smallest valid value = {}'.format(n, smallest_valid_n))
      print('Setting n to smallest valid value = {}'.format(smallest_valid_n))
      n = smallest_valid_n

  center_x = expand(re(init_fixed_point))
  center_y = expand(im(init_fixed_point) * (cot(angle1)*cot(angle2) + cos(pi/n)*csc(angle1)*csc(angle2)))

  radius_sq = expand( im(init_fixed_point)**2 * (-1 + (cot(angle1)*cot(angle2) + cos(pi/n)*csc(angle1)*csc(angle2))**2) )
  radius = radsimp(sqrt(radius_sq))

  return center_x, center_y , radius

In [ ]:
a,b,n = symbols('a b n', real=True)
ValidSecondFixedPoint(a+b*I, 5*pi/6, 5*pi/6, n=3)

(a, 5*b, 2*sqrt(6)*Abs(b))

In [ ]:
expand(((1-sqrt(3))/2 - (1+sqrt(3))/2)**2 + ((-1+sqrt(3))/2 - 5*((1+sqrt(3))/2))**2)

12*sqrt(3) + 24

In [ ]:
expand(((1+sqrt(3))/2)**2)*24

12*sqrt(3) + 24

In [ ]:
a, b, c, d, t, s, x, y = symbols('a b c d t s x y', real=True)

In [ ]:
A = simplify(MobiusPointToPoint(I, a+I*b) * Matrix([[cos(t), -sin(t)], [sin(t), cos(t)]]) * MobiusPointToPoint(I, a+I*b).inv())

In [ ]:
B = simplify(MobiusPointToPoint(I, x+I*y) * Matrix([[cos(s), -sin(s)], [sin(s), cos(s)]]) * MobiusPointToPoint(I, x+I*y).inv())

In [ ]:
expand(A*B).trace()

-a**2*sin(s)*sin(t)/(b*y) + 2*a*x*sin(s)*sin(t)/(b*y) - b*sin(s)*sin(t)/y + 2*cos(s)*cos(t) - x**2*sin(s)*sin(t)/(b*y) - y*sin(s)*sin(t)/b

In [ ]:
A_ = A.subs([(t,5*pi/6), (a, -1/2), (b,1/2)])

In [ ]:
B_ = B.subs([(t,5*pi/6), (c, 1/4), (d,1/4)])

In [ ]:
A_

Matrix([
[-sqrt(3)/2 - 0.5,            -0.5],
[             1.0, 0.5 - sqrt(3)/2]])

In [ ]:
B_

Matrix([
[0.5 - sqrt(3)/2,            -0.25],
[            2.0, -sqrt(3)/2 - 0.5]])

In [ ]:
A = psl_so(*Matrix([[cos(t), -sin(t)], [sin(t), cos(t)]])).subs([(t,pi/2)])

In [ ]:
A**2

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [ ]:
factor(psl_so(*Matrix([[cos(t), -sin(t)], [sin(t), cos(t)]])).subs([(t, pi/2)]).charpoly().as_expr())

(lambda - 1)*(lambda + 1)**2

In [ ]:
# Isometries in SO(1,2)
A = Matrix([[2, sqrt(3), 0], [0, 0, -1], [sqrt(3), 2, 0]])
B = Matrix([[2, -sqrt(3), 0], [0, 0, -1], [-sqrt(3), 2, 0]])

In [ ]:
# Convert to isometries in PSL(2,R)
T_A = so_psl(A)
T_B = so_psl(B)

In [ ]:
# Standard elliptic element that A is conjugate to
R_A = Matrix([[cos(5*pi/6), -sin(5*pi/6)], [sin(5*pi/6), cos(5*pi/6)]])

# Element that conjugates R_A to A
P_A = Matrix([[1 + sqrt(3), -1 - sqrt(3)], [2,0]])

In [ ]:
# Rational Canonical Form for A and B
RCF = CompanionMatrix(1,-2,2)

In [ ]:
RCF_A = ConjugatorToRCF(A)
RCF_B = ConjugatorToRCF(B)

In [ ]:
P_new = Matrix([[1,1],[0,1]])
T_B_new = P_new * R_A * P_new.inv()

In [ ]:
B_new = psl_so(*T_B_new)

In [ ]:
test = ConjugatorToRCF(B_new)

In [ ]:
so_psl(B_new)

Matrix([
[1/2 - sqrt(3)/2,               -1],
[            1/2, -sqrt(3)/2 - 1/2]])

Matrix([
[1/2 - sqrt(3)/2,               -1],
[            1/2, -sqrt(3)/2 - 1/2]])

In [ ]:
A = Elliptic(5*pi/6, 1 + sqrt(2)*I)
B = Elliptic(5*pi/6, -sqrt(3) + I)

In [ ]:
A.trace

Matrix([
[    -sqrt(6)/4 - 3*sqrt(2)/8 + 3/2, -sqrt(2)/2 + sqrt(3)],
[-sqrt(6)/4 - sqrt(3)/4 - sqrt(2)/8,           -sqrt(2)/2]])

In [ ]:
expand((A*B).trace())

-7*sqrt(2)/8 - sqrt(6)/4 + 3/2

## Scratch work

In [ ]:
T1 = so_psl(Matrix([[2, sqrt(3), 0], [0, 0, -1], [sqrt(3), 2, 0]]))

In [ ]:
T2 = so_psl(Matrix([[2, -sqrt(3), 0], [0, 0, -1], [-sqrt(3), 2, 0]]))

In [ ]:
T1

Matrix([
[ 1/2 - sqrt(3)/2, -sqrt(3)/2 - 1/2],
[-1/2 + sqrt(3)/2, -sqrt(3)/2 - 1/2]])

In [ ]:
psl_so(*T1).charpoly()

PurePoly(lambda**3 - 2*lambda**2 + 2*lambda - 1, lambda, domain='ZZ')

In [ ]:
psl_so(*T2).charpoly()

PurePoly(lambda**3 - 2*lambda**2 + 2*lambda - 1, lambda, domain='ZZ')

In [ ]:
A = Matrix([[2, sqrt(3), 0], [0, 0, -1], [sqrt(3), 2, 0]])

In [ ]:
B = Matrix([[2, -sqrt(3), 0], [0, 0, -1], [-sqrt(3), 2, 0]])

In [ ]:
A.charpoly()

PurePoly(lambda**3 - 2*lambda**2 + 2*lambda - 1, lambda, domain='ZZ')

In [ ]:
B.charpoly()

PurePoly(lambda**3 - 2*lambda**2 + 2*lambda - 1, lambda, domain='ZZ')

In [ ]:
B**2 - B

Matrix([
[       2, -sqrt(3), sqrt(3)],
[ sqrt(3),       -2,       1],
[-sqrt(3),        1,      -2]])

In [ ]:
A**2 - A

Matrix([
[       2, sqrt(3), -sqrt(3)],
[-sqrt(3),      -2,        1],
[ sqrt(3),       1,       -2]])

In [ ]:
P, M = A.jordan_form()

In [ ]:
P

Matrix([
[sqrt(3), 2*sqrt(3)*I/(sqrt(3) + 3*I), -2*sqrt(3)*I/(sqrt(3) - 3*I)],
[     -1,      1/(-1/2 + sqrt(3)*I/2),       1/(-1/2 - sqrt(3)*I/2)],
[      1,                           1,                            1]])

In [ ]:
radsimp(P[4])

(-1 - sqrt(3)*I)/2

In [ ]:
M

Matrix([
[1,                 0,                 0],
[0, 1/2 - sqrt(3)*I/2,                 0],
[0,                 0, 1/2 + sqrt(3)*I/2]])

In [ ]:
P = Matrix([[1 + sqrt(3), -1 - sqrt(3)], [2,0]])

In [ ]:
P = simplify(P/ sqrt(P.det()))

In [ ]:
A = Matrix([[cos(5*pi/6), -sin(5*pi/6)], [sin(5*pi/6), cos(5*pi/6)]])

In [ ]:
A = Matrix([[-sqrt(3)/2, Rational(-1,2)], [Rational(1,2), -sqrt(3)/2]])

In [ ]:
psl_so(*P).charpoly()

PurePoly(lambda**3 + (1/2 - sqrt(3)/2)*lambda**2 + (-1/2 + sqrt(3)/2)*lambda - 1, lambda, domain='EX')

In [ ]:
simplify(P*A*P.inv())

Matrix([
[ 1/2 - sqrt(3)/2, -sqrt(3)/2 - 1/2],
[-1/2 + sqrt(3)/2, -sqrt(3)/2 - 1/2]])

In [ ]:
P.inv()

Matrix([
[                 0, 1/2],
[-2/(2 + 2*sqrt(3)), 1/2]])

In [ ]:
B = Matrix([[(1-sqrt(3))/2, (-1-sqrt(3))/2], [(-1+sqrt(3))/2, (-1-sqrt(3))/2]])

In [ ]:
B

Matrix([
[ 1/2 - sqrt(3)/2, -sqrt(3)/2 - 1/2],
[-1/2 + sqrt(3)/2, -sqrt(3)/2 - 1/2]])

In [ ]:
B.eigenvals()

{-sqrt(3)/2 - I/2: 1, -sqrt(3)/2 + I/2: 1}

In [ ]:
v2 = expand(B.eigenvects()[0][2][0])
v1 = expand(B.eigenvects()[1][2][0])

In [ ]:
v1 + v2

Matrix([
[1 + sqrt(3)],
[          2]])

In [ ]:
expand(I*(v1 - v2))

Matrix([
[-sqrt(3) - 1],
[           0]])

In [ ]:
radsimp(((-sqrt(3)/2)*I - (1/2)) / ((1/2)*I - (sqrt(3)/2)))

1.0*I

In [ ]:
Mobius(-sqrt(3)/2, Rational(-1,2), Rational(1,2), -sqrt(3)/2, I)

I

In [ ]:
FixedPoint(-sqrt(3)/2, Rational(-1,2), Rational(1,2), -sqrt(3)/2)

(I, -I)

In [ ]:
FixedPoint(*B)[0]

(1 + sqrt(3) + I + sqrt(3)*I)/2

In [ ]:
Mobius(*P.inv(), FixedPoint(*B)[0])

I

In [ ]:
Mobius(*P, I)

1/2 + (sqrt(3) + I + sqrt(3)*I)/2

In [ ]:
Matrix([[1,2,4], [0,0,-sqrt(3)],[0, sqrt(3), 2*sqrt(3)]]) * Matrix([[0,0,1], [1,0,-2], [0,1,2]]) * Matrix([[1,2,4], [0,0,-sqrt(3)],[0, sqrt(3), 2*sqrt(3)]]).inv()

Matrix([
[      2, sqrt(3),  0],
[      0,       0, -1],
[sqrt(3),       2,  0]])

In [ ]:
a,b,c,d, x, y,t = symbols('a b c d x y t')

In [ ]:
solns = solve([a*x + b + c*(-x**2 + y**2) - d*x, a*y - 2*x*y*c - d*y, c*x + d - cos(t), c*y - sin(t)], a,b,c,d)

In [ ]:
a_ = solns[a]
b_ = solns[b]
c_ = solns[c]
d_ = solns[d]

In [ ]:
expand(psl_so(a_, b_, c_, d_))

Matrix([
[                          x**4*sin(t)**2/(2*y**2) + x**2*sin(t)**2 + x**2*sin(t)**2/y**2 + y**2*sin(t)**2/2 + cos(t)**2 + sin(t)**2/(2*y**2), -x**4*cos(2*t)/(4*y**2) + x**4/(4*y**2) - x**2*cos(2*t)/2 + x**2/2 - x*sin(2*t)/y - y**2*cos(2*t)/4 + y**2/4 + cos(2*t)/(4*y**2) - 1/(4*y**2), -x**3*sin(t)**2/y**2 - x**2*sin(t)*cos(t)/y - x*sin(t)**2 - x*sin(t)**2/y**2 - y*sin(t)*cos(t) + sin(t)*cos(t)/y],
[x**4*cos(2*t)/(4*y**2) - x**4/(4*y**2) + x**2*cos(2*t)/2 - x**2/2 - x*sin(2*t)/y + y**2*cos(2*t)/4 - y**2/4 - cos(2*t)/(4*y**2) + 1/(4*y**2),                           -x**4*sin(t)**2/(2*y**2) - x**2*sin(t)**2 + x**2*sin(t)**2/y**2 - y**2*sin(t)**2/2 + cos(t)**2 - sin(t)**2/(2*y**2),  x**3*sin(t)**2/y**2 + x**2*sin(t)*cos(t)/y + x*sin(t)**2 - x*sin(t)**2/y**2 + y*sin(t)*cos(t) + sin(t)*cos(t)/y],
[                             x**3*sin(t)**2/y**2 - x**2*sin(t)*cos(t)/y + x*sin(t)**2 + x*sin(t)**2/y**2 - y*sin(t)*cos(t) + sin(t)*cos(t)/y,                               x**3*sin(t)**2/y

In [ ]:
T_A

Matrix([
[ 1/2 - sqrt(3)/2, -sqrt(3)/2 - 1/2],
[-1/2 + sqrt(3)/2, -sqrt(3)/2 - 1/2]])

In [ ]:
FixedPoints(*T_A)[0]

(1 + sqrt(3) + I + sqrt(3)*I)/2

In [ ]:
expand(Matrix([[a_, b_], [c_, d_]]).subs([(x, (1+sqrt(3))/2), (y, (1+sqrt(3))/2), (t,5*pi/6)]))

Matrix([
[1/2 - sqrt(3)/2, -sqrt(3)/2 - 1/2],
[1/(1 + sqrt(3)), -sqrt(3)/2 - 1/2]])

In [ ]:
psl_so(*expand(Matrix([[a_, b_], [c_, d_]]).subs([(x, 0), (y, 1), (t,pi/5)]))).charpoly()

PurePoly(lambda**3 + (-sqrt(5)/2 - 1/2)*lambda**2 + (1/2 + sqrt(5)/2)*lambda - 1, lambda, domain='EX')